In [29]:
len(df)


8036

In [1]:
import pickle
import numpy as np
import math

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Flatten, Dropout


Using TensorFlow backend.


In [36]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
%matplotlib inline
def preprocess(img):
    img = img[40:120,:]
    img = cv2.resize(img,(64,64))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
    return img
img=mpimg.imread('data/IMG/center_2016_12_01_13_30_48_287.jpg')
img2=preprocess(img)
plot_image = np.concatenate((img,img2))
print(img.shape)
#img = cv2.flip(preprocess(img),1)
plt.imshow(plot_image)
print(img.shape)


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [24]:
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import pickle
import random
df = pd.read_csv('data/driving_log.csv')
#rand_locs = np.random.choice(len(df['steering']),5)
#df_filtered = df.loc[rand_locs,:]
df_filtered = df
X = []
y = []
path = 'data/'
STEERING_OFFSET = 0.25
mn = np.mean(df_filtered['steering'])
std = np.std(df_filtered['steering'])
for i in range(len(df_filtered)):
        c = mpimg.imread(path + df_filtered.iloc[i,:]['center'])
        l = mpimg.imread(path + df_filtered.iloc[i,:]['left'][1:])
        r = mpimg.imread(path + df_filtered.iloc[i,:]['right'][1:])
        num = np.random.normal(mn,std)
        if not ((abs(df_filtered.iloc[i,:]['steering']) < 0.1) and df_filtered.iloc[i,:]['steering'] < num): 
            X.append(c)
            X.append(cv2.flip(c,1))
            y.append(df_filtered.iloc[i,:]['steering'])
            y.append(-1.*df_filtered.iloc[i,:]['steering'])
            X.append(l)
            X.append(cv2.flip(l,1))
            X.append(r)
            X.append(cv2.flip(r,1))
            y.append(df_filtered.iloc[i,:]['steering'] + STEERING_OFFSET)
            y.append(-1.*(df_filtered.iloc[i,:]['steering'] + STEERING_OFFSET))
            y.append(df_filtered.iloc[i,:]['steering'] - STEERING_OFFSET)
            y.append(-1.*(df_filtered.iloc[i,:]['steering'] - STEERING_OFFSET))
for i, el in enumerate(X):
    X[i] = preprocess(el)
X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2) 


In [51]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Flatten, Dropout


model = Sequential()
model.add(BatchNormalization(input_shape=X[0].shape))
model.add(Conv2D(24, 5, 5,subsample=(2,2), dim_ordering='tf'))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Conv2D(36, 5, 5,subsample=(2,2) ,dim_ordering='tf'))
model.add(Activation('elu'))
model.add(Conv2D(48, 5, 5,subsample=(2,2), dim_ordering='tf'))
model.add(Activation('elu'))
model.add(Conv2D(64, 3, 3, dim_ordering='tf'))
model.add(Activation('elu'))
model.add(Conv2D(64, 3, 3, dim_ordering='tf'))
model.add(Activation('elu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('elu'))
model.add(Dense(50))
model.add(Activation('elu'))
model.add(Dense(10))
model.add(Activation('tanh'))
model.add(Dense(1))
model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()




____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_3 (BatchNorma (None, 64, 64, 3)     12          batchnormalization_input_3[0][0] 
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 30, 30, 24)    1824        batchnormalization_3[0][0]       
____________________________________________________________________________________________________
activation_17 (Activation)       (None, 30, 30, 24)    0           convolution2d_11[0][0]           
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 30, 30, 24)    0           activation_17[0][0]              
___________________________________________________________________________________________

In [39]:
from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical
import GraphViz

ImportError: No module named 'GraphViz'

In [25]:
import json
imageGen = ImageDataGenerator()
history = model.fit_generator(imageGen.flow(X_train, y_train, batch_size=256), samples_per_epoch=len(X_train),
                              nb_epoch=5, validation_data=imageGen.flow(X_val, y_val, batch_size=256),
                              nb_val_samples=len(X_val))

model.save_weights("model.h5", True)
with open('model.json', 'w') as outfile:
    json.dump(model.to_json(), outfile)

Epoch 1/5
26496/26496 [==============================] - 31s - loss: 0.0194 - acc: 0.1280 - val_loss: 0.0176 - val_acc: 0.1220
Epoch 2/5
26496/26496 [==============================] - 29s - loss: 0.0188 - acc: 0.1279 - val_loss: 0.0181 - val_acc: 0.1212
Epoch 3/5
26496/26496 [==============================] - 29s - loss: 0.0186 - acc: 0.1279 - val_loss: 0.0167 - val_acc: 0.1217
Epoch 4/5
26496/26496 [==============================] - 29s - loss: 0.0177 - acc: 0.1280 - val_loss: 0.0167 - val_acc: 0.1261
Epoch 5/5
26496/26496 [==============================] - 29s - loss: 0.0172 - acc: 0.1279 - val_loss: 0.0163 - val_acc: 0.1196


In [43]:
!conda install GraphViz -y

Fetching package metadata .......
Solving package specifications: ..........

Package plan for installation in environment /home/carnd/anaconda3/envs/carnd-term1:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libtool-2.4.2              |                0         547 KB
    pillow-4.0.0               |           py35_0         863 KB
    pango-1.39.0               |                0         777 KB
    graphviz-2.38.0            |                4        12.0 MB
    qt-4.8.7                   |                4        32.7 MB
    matplotlib-1.5.1           |      np111py35_0         8.4 MB
    ------------------------------------------------------------
                                           Total:        55.2 MB

The following NEW packages will be INSTALLED:

    expat:      2.1.0-0                      
    graphviz:   2.38.0-4                     
    libtool:    2.4.2-0         

In [50]:
!conda install pydot

Fetching package metadata .......
Solving package specifications: ....
UnsatisfiableError: The following specifications were found to be in conflict:
  - pydot
  - python 3.5*
Use "conda info <package>" to see the dependencies for each package.


